# Training PPO Model on CarRacing-v3 Environment

This notebook demonstrates how to train a Proximal Policy Optimization (PPO) agent with a CNN policy on the CarRacing-v3 environment from Gymnasium using vectorized environments for faster training.

## 1. Import Required Libraries

Import the necessary libraries, including Gymnasium, Stable-Baselines3, and other dependencies for vectorized environments and PPO training.

In [2]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecFrameStack
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
import os
import numpy as np
import torch

print(f"Gymnasium version: {gym.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Gymnasium version: 1.2.2
PyTorch version: 2.6.0+cu124
CUDA available: True


## 2. Create Vectorized CarRacing-v3 Environments

Use Gymnasium's make_vec_env to create multiple parallel instances of the CarRacing-v3 environment for faster training.

In [3]:
# Configuration parameters
NUM_ENVS = 6  # Number of parallel environments
ENV_ID = "CarRacing-v3"

# Training configuration
TOTAL_TIMESTEPS = 2_000_000  # Total number of timesteps to train
SAVE_FREQ = 50_000  # Save model every N steps

# Create vectorized environments
# Using SubprocVecEnv for true parallelization (separate processes)
# Use DummyVecEnv if you encounter issues with SubprocVecEnv
env = make_vec_env(
    ENV_ID,
    n_envs=NUM_ENVS,
    vec_env_cls=SubprocVecEnv,
    seed=42
)

# Optional: Stack frames for temporal information
# env = VecFrameStack(env, n_stack=4)

print(f"Created {NUM_ENVS} vectorized environments")
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

Created 6 vectorized environments
Observation space: Box(0, 255, (96, 96, 3), uint8)
Action space: Box([-1.  0.  0.], 1.0, (3,), float32)


## 3. Initialize PPO Model with CNN Policy

Create a PPO model from Stable-Baselines3 with a CnnPolicy, which is suitable for processing the visual observations from the CarRacing environment.

In [5]:
# Create directories for saving models and logs
os.makedirs("base_models", exist_ok=True)
os.makedirs("logs", exist_ok=True)

# Default PPO hyperparameters
model = PPO(
    policy="CnnPolicy",
    env=env,
    learning_rate=3e-4,
    n_steps=2048,  # Number of steps to run for each environment per update
    batch_size=64,  # Minibatch size
    n_epochs=10,  # Number of epoch when optimizing the surrogate loss
    gamma=0.99,  # Discount factor
    verbose=1,
    tensorboard_log="./logs/",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

print(f"Model initialized with device: {model.device}")
print(f"Policy architecture: CnnPolicy")

Using cuda device
Wrapping the env in a VecTransposeImage.
Model initialized with device: cuda
Policy architecture: CnnPolicy


## 4. Train the PPO Model

Train the PPO model on the vectorized environments using the learn() method with specified timesteps and logging parameters.

In [6]:


# Setup callbacks
checkpoint_callback = CheckpointCallback(
    save_freq=SAVE_FREQ // NUM_ENVS,  # Adjust for vectorized envs
    save_path="./base_models/checkpoints/",
    name_prefix="ppo_carracing"
)

# Optional: Create evaluation environment for monitoring progress
eval_env = make_vec_env(ENV_ID, n_envs=1, seed=100)
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./base_models/best_model/",
    log_path="./logs/eval/",
    eval_freq = max(SAVE_FREQ // NUM_ENVS, 1),  # Evaluate every N steps
    n_eval_episodes=5,
    deterministic=True,
    render=False
)


<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
c:\Users\migue\anaconda3\envs\rl-racing\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [ ]:

# Train the model
print("Starting training...")
model.learn(
    total_timesteps=TOTAL_TIMESTEPS,
    callback=[checkpoint_callback, eval_callback],
    progress_bar=True
)

print("Training completed!")

## 4.2 Restore a checkpoint

In [7]:
import glob
import os
import re

# Find the latest checkpoint in the checkpoints directory
checkpoint_dir = "./base_models/checkpoints/"
checkpoint_files = glob.glob(os.path.join(checkpoint_dir, "ppo_carracing_*.zip"))

if checkpoint_files:
    # Sort by modification time to get the latest
    latest_checkpoint = max(checkpoint_files, key=os.path.getmtime)
    print(f"Loading checkpoint: {latest_checkpoint}")
    
    # Extract the step number from the checkpoint filename
    # Format: ppo_carracing_XXXXXX_steps.zip
    match = re.search(r'ppo_carracing_(\d+)_steps\.zip', latest_checkpoint)
    if match:
        current_steps = int(match.group(1))
        print(f"Checkpoint is at step: {current_steps}")
        
        # Calculate remaining steps to reach TOTAL_TIMESTEPS
        remaining_steps = TOTAL_TIMESTEPS - current_steps
        
        if remaining_steps <= 0:
            print(f"Training already completed! Current steps ({current_steps}) >= Target ({TOTAL_TIMESTEPS})")
        else:
            print(f"Remaining steps to train: {remaining_steps}")
            
            # Create NEW vectorized environment with different number of workers
            NUM_ENVS_RESUME = 4  # Different from original NUM_ENVS
            env_resume = make_vec_env(
                ENV_ID,
                n_envs=NUM_ENVS_RESUME,
                vec_env_cls=SubprocVecEnv,
                seed=42
            )
            print(f"Created {NUM_ENVS_RESUME} vectorized environments for resumed training")
            
            # Load the model from checkpoint with NEW environment
            model = PPO.load(latest_checkpoint, env=env_resume)
            print(f"Model loaded successfully from: {latest_checkpoint}")
            print(f"Resuming training with {NUM_ENVS_RESUME} workers...")
            
            # Update callbacks for new number of environments
            checkpoint_callback_resume = CheckpointCallback(
                save_freq=SAVE_FREQ // NUM_ENVS_RESUME,
                save_path="./base_models/checkpoints/",
                name_prefix="ppo_carracing"
            )
            
            eval_callback_resume = EvalCallback(
                eval_env,
                best_model_save_path="./base_models/best_model/",
                log_path="./logs/eval/",
                eval_freq=max(SAVE_FREQ // NUM_ENVS_RESUME, 1),
                n_eval_episodes=5,
                deterministic=True,
                render=False
            )
            
            # Continue training from this checkpoint for REMAINING steps only
            model.learn(
                total_timesteps=remaining_steps,  # Train only remaining steps
                callback=[checkpoint_callback_resume, eval_callback_resume],
                progress_bar=True,
                reset_num_timesteps=False  # Continue counting timesteps from checkpoint
            )
            
            print(f"Training completed! Total steps reached: {TOTAL_TIMESTEPS}")
            
            # Close the resumed environment
            env_resume.close()
    else:
        print("Could not extract step number from checkpoint filename.")
else:
    print("No checkpoint files found. Please train from scratch first.")


Loading checkpoint: ./models/checkpoints\ppo_carracing_1249950_steps.zip
Checkpoint is at step: 1249950
Remaining steps to train: 750050
Created 4 vectorized environments for resumed training
Wrapping the env in a VecTransposeImage.
Model loaded successfully from: ./models/checkpoints\ppo_carracing_1249950_steps.zip
Resuming training with 4 workers...
Logging to ./logs/PPO_2
Created 4 vectorized environments for resumed training
Wrapping the env in a VecTransposeImage.
Model loaded successfully from: ./models/checkpoints\ppo_carracing_1249950_steps.zip
Resuming training with 4 workers...
Logging to ./logs/PPO_2


Output()

c:\Users\migue\anaconda3\envs\rl-racing\Lib\site-packages\stable_baselines3\common\callbacks.py:418: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x000001E61E6DA180> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x000001E61EB8A2D0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | 5.05     |
| time/              |          |
|    fps             | 412      |
|    iterations      | 1        |
|    time_elapsed    | 19       |
|    total_timesteps | 1258142  |
---------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1e+03     |
|    ep_rew_mean          | 12.6      |
| time/                   |           |
|    fps                  | 335       |
|    iterations           | 2         |
|    time_elapsed         | 48        |
|    total_timesteps      | 1266334   |
| train/                  |           |
|    approx_kl            | 40.659527 |
|    clip_fraction        | 0.966     |
|    clip_range           | 0.2       |
|    entropy_loss         | 3.88      |
|    explained_variance   | 0.942     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.189     |
|    n_updates            | 1020      |
|    policy_gradient_loss | 0.0833    |
|    std                  | 0.066     |
|    value_loss           | 0.734     |
---------------------------------------


...

Eval num_timesteps=1999950, episode_reward=-72.00 +/- 7.43

Episode length: 1000.00 +/- 0.00

----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 1e+03      |
|    mean_reward          | -72        |
| time/                   |            |
|    total_timesteps      | 1999950    |
| train/                  |            |
|    approx_kl            | 0.60438097 |
|    clip_fraction        | 0.675      |
|    clip_range           | 0.2        |
|    entropy_loss         | 4.76       |
|    explained_variance   | 0.989      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0345    |
|    n_updates            | 1920       |
|    policy_gradient_loss | 0.00926    |
|    std                  | 0.0489     |
|    value_loss           | 0.0435     |
----------------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -66.1    |
| time/              |          |
|    fps             | 186      |
|    iterations      | 92       |
|    time_elapsed    | 4030     |
|    total_timesteps | 2003614  |
---------------------------------


Training completed! Total steps reached: 2000000


## 5. Save the Trained Model

Save the trained PPO model to disk for later evaluation or continued training.

In [8]:
# Save the final trained model
model_path = "./base_models/ppo_carracing_final"
model.save(model_path)
print(f"Model saved to: {model_path}")

# Save the vectorized environment statistics (if using VecNormalize)
# env.save("./models/vec_normalize.pkl")

# Close environments
env.close()
eval_env.close()

print("All environments closed successfully!")

Model saved to: ./models/ppo_carracing_final
All environments closed successfully!
All environments closed successfully!


## Next Steps

- To evaluate the trained model, load it using `PPO.load(model_path)`
- Use TensorBoard to visualize training progress: `tensorboard --logdir ./logs/`
- Fine-tune hyperparameters based on training curves
- Consider using frame stacking or action repeat for better performance

In [19]:
from gymnasium.wrappers import RecordVideo

# Create a new environment with human rendering
test_env = gym.make(ENV_ID, render_mode="rgb_array")
model_path = "./base_models/best_model/best_model.zip"

test_env = RecordVideo(
    test_env,
    video_folder="./videos/base",
    episode_trigger=lambda episode_id: True  # Record all episodes
)

# Load the trained model
model = PPO.load(model_path)

NUM_EPISODES = 10
for i in range(NUM_EPISODES):
    # Run the agent in the environment
    obs, info = test_env.reset()
    done = False
    total_reward = 0
    step_count = 0

    print(f"Starting to play Episode {i+1} with the trained agent...")
    print("Close the window to stop.")

    try:
        while not done:
            # Predict action using the trained model
            action, _states = model.predict(obs, deterministic=True)

            # Take action in the environment
            obs, reward, terminated, truncated, info = test_env.step(action)
            done = terminated or truncated

            total_reward += reward
            step_count += 1

            # Optional: print progress every 100 steps
            if step_count % 100 == 0:
                print(f"Step: {step_count}, Total Reward: {total_reward:.2f}")

        print(f"\nEpisode {i+1} finished!")
        print(f"Total steps: {step_count}")
        print(f"Total reward: {total_reward:.2f}")

    finally:
        test_env.close()

Starting to play Episode 1 with the trained agent...
Close the window to stop.
Step: 100, Total Reward: 0.03
Step: 100, Total Reward: 0.03
Step: 200, Total Reward: 0.07
Step: 200, Total Reward: 0.07
Step: 300, Total Reward: -9.93
Step: 300, Total Reward: -9.93
Step: 400, Total Reward: -13.24
Step: 400, Total Reward: -13.24
Step: 500, Total Reward: -13.21
Step: 500, Total Reward: -13.21
Step: 600, Total Reward: -23.21
Step: 600, Total Reward: -23.21
Step: 700, Total Reward: 47.06
Step: 700, Total Reward: 47.06
Step: 800, Total Reward: 87.22
Step: 800, Total Reward: 87.22
Step: 900, Total Reward: 77.22
Step: 900, Total Reward: 77.22
Step: 1000, Total Reward: 67.22

Episode 1 finished!
Total steps: 1000
Total reward: 67.22
Step: 1000, Total Reward: 67.22

Episode 1 finished!
Total steps: 1000
Total reward: 67.22
Starting to play Episode 2 with the trained agent...
Close the window to stop.
Starting to play Episode 2 with the trained agent...
Close the window to stop.
Step: 100, Total Rewa